## Step1 初始化 Jupyter 环境 & 导入包

In [ ]:
# 用于在 Jupyter 中强制刷新参数
%reset -f

# 导入相关的包
import os
import sys
from collections import deque
from pathlib import Path

import torch
import pygame
import imageio
import gymnasium as gym
import numpy as np

from tqdm.notebook import tqdm
from torch.distributions import Categorical
from torchvision.transforms import v2
from loguru import logger

## Step2 设置相关参数

In [ ]:
# 相关功能
is_training = 1                     # 是否进行训练
is_evaluate = 1                     # 是否进行评估, 此时会渲染游戏画面
need_record = 1                     # 是否开启录像, 前提是 is_evaluate=1 才有效, 不会渲染游戏画面

# 日志等级
log_level = "INFO"
logger.remove()
logger.add(sys.stderr, level=log_level)

# 环境信息
env_id = "CartPole-v1"              # 游戏环境名
max_steps = 500                     # 每个回合的最大步数
render_mode = "rgb_array"           # 渲染模式，可选 "human"、"rgb_array" 等

# Policy Gradient 算法参数
gamma = 0.95                        # 折扣因子
memory_buffer_size = 10000          # 记忆缓存区大小
frame_stack = 2                     # 帧堆叠的数量

# 训练参数
num_train_episodes = 1000           # 训练的总回合数
lr = 1e-3                           # 学习率

# 评估参数
num_eval_episodes = 10              # 评估的回合数
reward_threshold = 500              # 评估奖励阈值, 如果高于阈值时, 日志等级为 Success, 否则为 Warning
eval_sample_action = True           # 评估时的动作是否基于概率来采样, True 则基于概率来选取动作, False 则直接选取最大概率


# 保存策略
save_dir = "./Gym_Classic_CartPole_Policy_Gradient_FrameStack"   # 数据保存的目录
save_freq = 100                                                  # 模型保存的频率
max_checkpoints = 5                                              # 最大保存的模型数量
checkpoint_perfix = "CheckPoint_Gym_Classic_CartPole_FS_"        # 模型保存的前缀
evaluate_record_perfix = "Video_Gym_Classic_CartPole_FS_"        # 评估记录保存的前缀
evaluate_record_fps = 30                                         # 评估记录保存的帧率
evaluate_record_quality = 10                                     # 评估记录保存的质量, 值为 0 ~ 10

# 其余参数初始化
device = "cuda" if torch.cuda.is_available() else "cpu"

## Step3 预处理函数 & 工具

In [ ]:
def get_max_checkpoint_id(save_dir=save_dir, checkpoint_perfix=checkpoint_perfix):
    """
    获取最新的模型路径, 并返回 "模型路径" 和 checkpoint 对应的 id
    """
    # 如果指定目录不存在, 则直接创建该目录
    if not Path(save_dir).exists():
        Path(save_dir).mkdir(parents=True)
        logger.debug("The specified directory does not exist, will create this folder")
        return None
    
    # 获取所有的模型文件
    checkpoints = []
    current_path = Path(save_dir)
    for entry in current_path.iterdir():
        if entry.is_file() and entry.suffix == ".pth" and entry.name.startswith(checkpoint_perfix):
            id = entry.name.split(checkpoint_perfix)[-1].split(".")[0]
            checkpoints.append(id)
    
    # 寻找最大的 checkpoint id
    if checkpoints.__len__() == 0:
        logger.info("Not found any checkpoint files, will random initialization of network parameters")
        return None
    else:
        max_checkpoint_id = max(checkpoints)
        max_checkpoint_path = os.path.abspath(f"{save_dir}/{checkpoint_perfix}{max_checkpoint_id}.pth")
        logger.info(f"Found max checkpoints, max_checkpoint_id is {max_checkpoint_id}")
        return {"max_checkpoint_path" : max_checkpoint_path, "max_checkpoint_id" : max_checkpoint_id}

In [ ]:
def del_old_checkpoint(save_dir=save_dir, checkpoint_perfix=checkpoint_perfix, max_checkpoints=max_checkpoints):
    """
    删除旧的模型文件, 只保留最新的 max_checkpoints 个模型文件
    """
    if Path(save_dir).exists():
        checkpoints = []
        for entry in Path(save_dir).iterdir():
            if entry.is_file() and entry.suffix == ".pth" and entry.name.startswith(checkpoint_perfix):
                id = int(entry.name.split(checkpoint_perfix)[-1].split(".")[0])
                checkpoints.append(id)
    
    if checkpoints.__len__() > max_checkpoints:
        min_checkpoint_id = min(checkpoints)
        min_checkpoint_path = os.path.abspath(f"{save_dir}/{checkpoint_perfix}{min_checkpoint_id}.pth")
        os.remove(min_checkpoint_path)
        logger.warning(f"Delete old checkpoint file {min_checkpoint_path}")

## Step4 定义智能体

In [ ]:
class RLAgent:
    """
    智能体类, 封装了智能体所需要的各种方法
    """
    def __init__(self, action_size):
        # Global Args
        self.max_checkpoint = get_max_checkpoint_id()
        self.memory_buffer = deque(maxlen=memory_buffer_size)

        # Init Network
        self.network = torch.nn.Sequential(
            torch.nn.LazyLinear(out_features=128),
            torch.nn.LeakyReLU(),
            torch.nn.LazyLinear(out_features=128),
            torch.nn.LeakyReLU(),
            torch.nn.LazyLinear(out_features=action_size),
        )
        if self.max_checkpoint is not None:
            self.network.load_state_dict(torch.load(self.max_checkpoint["max_checkpoint_path"]))

        # Move to designated device
        self.network.to(device)

        # Transfoms
        self.transform = v2.Compose([
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Grayscale(1),
        ])

        # optimizer
        self.optimizer = torch.optim.AdamW(self.network.parameters(), lr=lr)

    def processing_states(self, frame_buffer):
        """
        对输入的 frame_buffer 进行预处理, 并返回模型可以处理的 Tensor 对象
        """
        # 将形状处理为 [1, size * frame_buffer_size]
        states = torch.stack(tuple(self.transform(frame_buffer)), dim=0)
        states = states.reshape(1, -1)
        logger.debug(f"Processing states shape: {states.shape}")
        return states
    
    def select_action(self, state, sample=True):
        """
        选择动作, 某些算法需要对模型的输出进行采样, 因此可以将 sample 设置为 True
        :param state:  神经网络可以接收的输入形状: [batch_size, color_channel * stack_size, height, width]
        :param sample: 动作是否是采样, 如果不是则直接选择概率最高的动作
        """
        state = state.to(device)
        if sample:
            # https://pytorch.ac.cn/docs/stable/distributions.html#categorical
            # 采样 & 动作的对数概率最好采用这种方法, 可以避免梯度消失的问题
            logits = self.network(state)
            action_dist = Categorical(logits=logits)
            action = action_dist.sample()
            log_prob = action_dist.log_prob(action)
            return {"action": action, "log_prob": log_prob}
        else:
            action_logits = self.network(state)
            action = action_logits.argmax(dim=1).item()
            return {"action": action}

    def update_policy(self):
        """
        更新 policy 参数
        """
        G_t = 0
        G_t_deque = deque()
        num_mems = len(self.memory_buffer)
        # 计算每一步的回报 G_t
        for i in range(num_mems)[::-1]:
            G_t = G_t * gamma + self.memory_buffer[i]["Rt"]
            G_t_deque.appendleft(G_t)
        logger.debug(f"G_t values: {G_t_deque}")
        # 对 G_t 进行标准化
        G_stand = torch.tensor(G_t_deque)
        eps = np.finfo(np.float32).eps.item()
        G_stand = ((G_stand - G_stand.mean()) / (G_stand.std() + eps)).to(device)
        # 计算梯度误差
        # 这里的期望是让 log_prob * G_stand 最大化, 即回报的奖励值最大化
        # 但由于梯度更新的反向传播(backward) 会让 log_prob * G_stand 最小化, 因此这里取负号 (-log_prob * G_stand), 这样效果是等价的
        policy_loss = []
        for i in range(num_mems):
            log_prob = self.memory_buffer[i]["log_prob"]
            policy_loss.append(-log_prob * G_stand[i])
        # 更新梯度信息(使用了梯度累积, 将所有样本的梯度信息累加起来, 然后再进行反向传播和参数更新)
        # 实测这里也可以写为 torch.cat(policy_loss).sum().to(device)
        policy_loss = torch.stack(policy_loss).sum().to(device)
        logger.info(f"Policy_Loss: {policy_loss.item():4f}")
        # 更新策略网络参数
        self.optimizer.zero_grad()
        policy_loss.backward()
        self.optimizer.step()
        # 清空经验池中的数据
        self.memory_buffer.clear()
    
    def save_model(self, episodes):
        """
        保存模型到指定路径, 并根据实际情况删除老的模型
        """
        # 没有任何已存在的模型文件, 即首次启动训练
        if self.max_checkpoint is None:
            max_checkpoint_path = os.path.abspath(f"{save_dir}/{checkpoint_perfix}{episodes}.pth")
        # 已存在模型文件的情况
        else:
            max_checkpoint_path = os.path.abspath(f"{save_dir}/{checkpoint_perfix}{episodes + int(self.max_checkpoint["max_checkpoint_id"])}.pth")

        torch.save(self.network.state_dict(), max_checkpoint_path)
        logger.info(f"Model saved to {max_checkpoint_path}")
        # 删掉老模型
        del_old_checkpoint()

## Step5 调整环境

In [ ]:
# 定制环境
class CustomEnv(gym.Wrapper):
    """
    定制环境, 继承自 gym.Wrapper 类, 用于修改环境的行为或奖励机制
    """
    def __init__(self, env):
        super().__init__(env)

    def reset(self):
        """
        重置环境
        """
        # 重置观察结果
        observation = self.env.reset()

        return observation
    
    def step(self, action):
        """
        执行动作, 并调整了env 的行为或奖励机制
        """
        # 调用原始环境的 step 方法
        observation, reward, terminated, truncated, info = self.env.step(action)

        # 返回最终结果: observation, reward, terminated, truncated, info
        return observation, reward, terminated, truncated, info


## Step6 训练智能体

In [ ]:
if is_training:
    # 训练用的主环境
    env = gym.make(env_id, render_mode=render_mode)
    env = CustomEnv(env)

    # 实例化智能体 (动作空间必须是离散的)
    if isinstance(env.action_space, gym.spaces.Discrete):
        action_size = env.action_space.n
        Agent = RLAgent(action_size=action_size)
    else:
        logger.error("Action space is not Discrete!")
        raise ValueError("Action space is not Discrete!")

    # 循环每个回合
    for episode in tqdm(range(num_train_episodes)):
        # 初始化环境
        state, info = env.reset()
        steps = 0
        total_reward = 0
        frame_buffer = deque(maxlen=frame_stack)
        current_action = None
        
        # 初始化帧缓冲区
        for _ in range(frame_stack): 
            frame_buffer.append(state)

        # 回合中的每一步
        for step in range(max_steps):
            # 处理当前状态
            current_states = Agent.processing_states(frame_buffer)
            # 选择动作 & 对数概率
            if step % frame_stack == 0:
                output = Agent.select_action(current_states)
                action, log_prob = output['action'].item(), output['log_prob']
                current_action = action
                logger.debug(f"Selected action: {action}")
            # 执行动作
            observation, reward, terminated, truncated, info = env.step(current_action)
            total_reward += reward
            logger.debug(f"Step {step + 1} | Reward: {reward} | Total Reward: {total_reward} | Terminated: {terminated} | Truncated: {truncated} | Info: {info}")
            # 更新帧缓冲区
            frame_buffer.append(observation)
            # 保存到记忆区
            if step % frame_stack == 0:
                Agent.memory_buffer.append({"St": current_states, "At": current_action, "Rt": reward, "log_prob": log_prob})
            # 如果该动作不需要决策动作, 则叠加奖励
            else:
                Agent.memory_buffer[-1]["Rt"] += reward
            # 判断是否结束该回合
            if terminated or truncated:
                if total_reward >= reward_threshold:
                    logger.success(f"Episode finish, total step {step + 1} | Total Reward: {total_reward}")
                else:
                    logger.warning(f"Episode finish, total step {step + 1} | Total Reward: {total_reward}")
                total_reward = 0
                break
        
        # 更新模型
        Agent.update_policy()
        
        # 保存模型
        if (episode + 1) % save_freq == 0 and episode != 0:
            episodes = episode + 1
            Agent.save_model(episodes)       

## Step7 评估智能体

In [ ]:
# 评估但不录制视频
if is_evaluate == 1 and need_record == 0:
    eval_env = gym.make(env_id, render_mode="human")
    eval_env = CustomEnv(eval_env)
# 评估且需要录制视频
elif is_evaluate == 1 and need_record == 1:
    eval_env = gym.make(env_id, render_mode="rgb_array")
    eval_env = CustomEnv(eval_env)

# 如果启用了评估
if is_evaluate == 1:

    # 初始化用于评估的参数
    frame_record = []
    max_reward = 0

    # 实例化用于评估的智能体
    Agent = RLAgent(action_size=eval_env.action_space.n)

    # 每个回合
    for episode in tqdm(range(num_eval_episodes)):
        # 初始化环境
        state, info = eval_env.reset()
        steps = 0
        total_reward = 0
        frame_buffer = deque(maxlen=frame_stack)
        current_action = None
        # 初始化帧缓冲区
        for _ in range(frame_stack): 
            frame_buffer.append(state)
            
        # 回合中的每一步
        for step in range(max_steps):
            # 处理当前状态
            current_states = Agent.processing_states(frame_buffer)
            # 选择动作
            if step % frame_stack == 0:
                output = Agent.select_action(current_states, sample=eval_sample_action)
                current_action = output["action"].item()
            # 执行该动作
            observation, reward, terminated, truncated, info = eval_env.step(current_action)
            total_reward += reward
            # 更新帧缓冲区
            frame_buffer.append(observation)
            # 如果需要记录视频, 则渲染画面 eval_env.render(), 然后将此画面添加到 frame_record 中
            if need_record:
                frame_record.append(eval_env.render())
                
            # 判断是否结束
            if terminated or truncated:
                # 如果需要记录视频, 则保留最好的记录
                if need_record and total_reward > max_reward:
                    np_frame_record = np.array(frame_record)
                    max_reward = total_reward
                    frame_record.clear()
                # 评估奖励
                if total_reward >= reward_threshold:
                    logger.success(f"Step {step + 1} | Total Reward: {total_reward}")
                else:
                    logger.warning(f"Step {step + 1} | Total Reward: {total_reward}")
                break

    # 记录评估结果(只记录最好的奖励轮次)
    if need_record:
        record_file = f"{os.path.abspath(os.path.join(save_dir, evaluate_record_perfix))}{int(max_reward)}.mp4"
        imageio.mimsave(record_file, np_frame_record, fps=evaluate_record_fps, quality=evaluate_record_quality)
        logger.info(f"The best evaluation record is: {record_file}")

    # 关闭环境
    eval_env.close()
    pygame.quit()